In [ ]:
import pandas as pd
import numpy as np
import re
import os
import requests
from bs4 import BeautifulSoup


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

dataset_path = '/content/drive/MyDrive/DataMan_Songs/Dataset/'

In [ ]:
# ## Sorted by popularity - all tuning
# https://www.songsterr.com/a/wa/all?r=tuning&lyrics=any&tuning=any&diff=any&inst=gtr&sort=p&vocals=any
# ## Sorted by popularity - standard1 tuning
# https://www.songsterr.com/a/wa/all?r=tuning&lyrics=any&tuning=std&diff=any&inst=gtr&sort=p&vocals=any
# ## Sorted by popularity - standard2 tuning
# https://www.songsterr.com/a/wa/all?r=tuning&lyrics=any&tuning=stdbass&diff=any&inst=gtr&sort=p&vocals=any
    
## Get list of all tunes
URL = "https://www.songsterr.com/a/wa/all?r=tuning&lyrics=any&tuning=any&diff=any&inst=gtr&sort=p&vocals=any"

page = requests.get(URL)

soup = BeautifulSoup(page.content, "html.parser")
# print(soup)
results = soup.find("div", id="FilterPanel")
# print(results)
tuning = results.find("select", id="tuning_any")
# print(tuning)

ls = str(tuning).split('value')

tunes = []
for el in ls[1:]:    
    a = re.findall(r"(\w+)", el)
    if a:
        tunes.append(a[0])
        
tunes


['any',
 'std',
 'stdbass',
 'hsd',
 'dropd',
 'dropc',
 'd',
 'dropcsharp',
 'dropdbass',
 'dropb',
 'hsdbass',
 'std7',
 'std5',
 'c',
 'csharp',
 'dropcbass',
 'b',
 'dbass',
 'dropasharp',
 'dropa',
 'openc']

In [ ]:
## Build a list of all site page with different tuning and sorted by popularity
link_p = 'https://www.songsterr.com/a/wa/all?r=tuning&lyrics=any&tuning=any&diff=any&inst=gtr&sort=p&vocals=any'

tune_site_p = {}
link_el = link_p.split('&')

for tune in tunes:
    site = link_el[0] + '&' + link_el[1] + '&' + 'tuning={}'.format(tune) + '&' + link_el[3] + '&' + link_el[4] + '&' + link_el[5] + '&' + link_el[6]
    tune_site_p[tune] = site

## Do the same thing sorting by date added
link_d = 'https://www.songsterr.com/a/wa/all?r=tuning&lyrics=any&tuning=any&diff=any&inst=gtr&sort=d&vocals=any'

tune_site_d = {}
link_el = link_d.split('&')

for tune in tunes:
    site = link_el[0] + '&' + link_el[1] + '&' + 'tuning={}'.format(tune) + '&' + link_el[3] + '&' + link_el[4] + '&' + link_el[5] + '&' + link_el[6]
    tune_site_d[tune] = site

tune_site_d['std']

'https://www.songsterr.com/a/wa/all?r=tuning&lyrics=any&tuning=std&diff=any&inst=gtr&sort=d&vocals=any'

In [ ]:
def song_link(tune_dict):

    song_dict = {key: [] for key in tune_dict.keys()}
    
    for key in tune_dict.keys(): 
       
#         print(key)
#         print(tune_dict[key])
        
    ## List songs available
        URL = tune_dict[key]

        page = requests.get(URL)

        soup = BeautifulSoup(page.content, "html.parser")
        results = soup.find("div", class_="listOuterWrapper")

        songs = results.find_all("a", class_="tab-link")

#         print(song_dict[tune])
        for song in songs:    
            song_dict[key].append(song['href'])

        print('len:' + str(len(song_dict[key])))
        
    ## Build the official link 'https://www.songsterr.com/' + song_path
        for path in song_dict[key]:

            # find the index of path
            i = song_dict[key].index(path)

            # replace path with 'https://www.songsterr.com/' + path
            song_dict[key] = song_dict[key][:i]+['https://www.songsterr.com' + str(path)]+song_dict[key][i+1:]
        
#         print('Name tune:' + str(key))
        
        dt = pd.DataFrame(song_dict[key])
#         dt.to_csv('Songs/SongLinks_P_{}.csv'.format(str(key)))  ## For song sorted by popularity
        dt.to_csv(dataset_path + '1_Links_Backup/SongLinks_D_{}.csv'.format(str(key)))  ## For song sorted by date added
    
    return()

In [ ]:
song_link(tune_site_p)

len:371
len:274
len:404
len:204
len:205
len:179
len:200
len:210
len:355
len:207
len:248
len:261
len:404
len:200
len:175
len:147
len:237
len:168
len:161
len:185
len:282


()

In [ ]:
os.chdir('Songs')

song_dt_concat_P = pd.DataFrame()
song_dt_concat_D = pd.DataFrame()

for song in os.listdir():
    if 'SongLinks_P_' in song:
#         print('Popularity')
        a = pd.read_csv(song, index_col='Unnamed: 0')
        song_dt_concat_P = pd.concat([song_dt_concat_P, a])
    else:
#         print('Date added')
        b = pd.read_csv(song, index_col='Unnamed: 0')
        song_dt_concat_D = pd.concat([song_dt_concat_D, b])


os.chdir('..')

song_dt_concat_P.to_csv(dataset_path + '1_Links_Bakcup/Song_links_P_Final.csv')
print('Lenght dataset popularity:' + str(len(song_dt_concat_P)))
print('Lenght dataset popularity clean:' + str(len(song_dt_concat_P.drop_duplicates())))

song_dt_concat_D.to_csv(dataset_path + '1_Links_Bakcup/Song_links_D_Final.csv')
print('Lenght dataset date added:' + str(len(song_dt_concat_D)))
print('Lenght dataset date added clean:' + str(len(song_dt_concat_D.drop_duplicates())))

song_dt_Final = pd.concat([song_dt_concat_P, song_dt_concat_D])
print('Lenght dataset final:' + str(len(song_dt_Final)))
song_dt_Final.drop_duplicates(inplace = True)
print('Lenght dataset final clean:' + str(len(song_dt_Final)))

song_dt_Final = song_dt_Final.rename(columns={'0': 'Link'})
song_dt_Final.to_csv(dataset_path + '1_Links_Bakcup/Song_dt_Final.csv')



Lenght dataset popularity:5076
Lenght dataset popularity clean:4710
Lenght dataset date added:4884
Lenght dataset date added clean:4728
Lenght dataset final:9960
Lenght dataset final clean:7312


In [ ]:
def song_features(dataset_file_name):

    df = pd.read_csv(dataset_file_name, index_col = 'Unnamed: 0')
    
## Search every link and extract informations
    band = []
    song = []
    player = []
    technique = []
    difficulty = []
    difficulty_score = []

    for link in df['Link']:

            print('-------------------------------------------------------------')
            print('Link: {}'.format(link))


        ##  Open link page
            URL = str(link)
            page = requests.get(URL)

        ##  Parse it with BeautifulSoup
            soup = BeautifulSoup(page.content, "html.parser")
            results = soup.find(id="header")

        ##  Find and add informations about the band and title song
            try:    
                Band_Song = results.find("h1", class_="C612su")
    #             print(Band_Song.text)
            except:
                print('\u0332'.join('Band and Song not found'))
                continue

            try: 
                band_song = (Band_Song.text).split('\xa0-\xa0')
    #             print(band_song)
            except:
    #             print('\u0332'.join('Band and Song split error'))
                break

            try:
                band.append(band_song[0])
            except:
                band.append(np.nan)
    #             print('\u0332'.join('Band not found'))
            try:
                song.append(band_song[1].replace(' Tab', ''))
            except:
                song.append(np.nan)
    #             print('\u0332'.join('Song not found'))


            ##  Find and add informations about the player and the technique
            try:
                Player_style = results.find("h2", class_="C61a4")
    #             print(Player_style.text)
            except:
    #             print('\u0332'.join('Player and Style not found'))
                continue

            try:
                player_list = (Player_style.text).replace('Difficulty (Rhythm):', '').replace('\xa0-\xa0', ' - ').split(' - ')
    #             print(player_list)

            except:
    #             print('\u0332'.join('Player and Style split error'))
                break

            try:
                if 'pachelbel' in link:
                    player.append(np.nan)
                    technique.append(player_list)
                else:
                    player.append(player_list[0].replace('Track: ', ''))
            except:
                player.append(np.nan)
    #             print('\u0332'.join('Player not found'))

            if 'pachelbel' not in link:
                try:
                    style = player_list[1:]
                    technique.append(style)
                except:
                    technique.append(np.nan)
    #                 print('\u0332'.join('Technique not found'))


            ##  Find and add informations about the track difficulty
            try:
                track_difficulty = soup.find(id="track-difficulty")
    #             print(track_difficulty)
                try:
                    diff = re.findall('(\w+)', str(track_difficulty))
                    index_pre_diff = diff.index('title')
    #                 print(diff)
    #                 print(index_pre_diff)

                    if diff[index_pre_diff + 1] == 'Beginner':
                        difficulty_score.append(diff[index_pre_diff + 3] + '/' + '8')
    #                     print('Success')

                    if diff[index_pre_diff + 1] == 'Intermediate':
                        difficulty_score.append(str(int(diff[index_pre_diff + 3]) + 2) + '/' + '8')
    #                     print('Success')

                    if diff[index_pre_diff + 1] == 'Advanced':
                        difficulty_score.append(str(int(diff[index_pre_diff + 3]) + 5) + '/' + '8')
    #                     print('Success')


                    difficulty.append(diff[index_pre_diff + 1])
                except:
                    difficulty_score.append(np.nan)
                    difficulty.append(np.nan)
    #                 print('Re operation findall error')
                    continue

            except:
    #             print('\u0332'.join('Diffulty not found'))
                difficulty.append(np.nan)
    #             n_errors += 1
                continue
        
    print('Scraping completed!')
    
    dataset = {'Band': band, 'Song': song, 
    'Player': player, 'Technique': technique,
    'Difficulty': difficulty,
    'Score': difficulty_score}
    
    print('Lenght passed: ' + str(len(df)))
    print('')
    
    for key in dataset.keys():
        print('Lenght {}: '.format(str(key)) + str(len(dataset[key])))
     
    dt = pd.DataFrame(dataset)
    dt.to_csv(dataset_path + 'Dataset_Songs_Features.csv')
    
    return(dt)

In [ ]:
song_features(dataset_path + '1_Links_Backup/Song_dt_Final.csv')

-------------------------------------------------------------
Link: https://www.songsterr.com/a/wsa/beatles-blackbird-tab-s57t0
-------------------------------------------------------------
Link: https://www.songsterr.com/a/wsa/isaac-albeniz-asturias-leyenda-tab-s23643t0
-------------------------------------------------------------
Link: https://www.songsterr.com/a/wsa/ac-dc-back-in-black-tab-s1024t2
-------------------------------------------------------------
Link: https://www.songsterr.com/a/wsa/pachelbel-canon-in-d-major-tab-s59950t0
-------------------------------------------------------------
Link: https://www.songsterr.com/a/wsa/led-zeppelin-babe-i-m-gonna-leave-you-tab-s6732t2
-------------------------------------------------------------
Link: https://www.songsterr.com/a/wsa/jimi-hendrix-little-wing-tab-s321t0
-------------------------------------------------------------
Link: https://www.songsterr.com/a/wsa/pink-floyd-shine-on-you-crazy-diamond-parts-i-ix-tab-s15518t1
--------

-------------------------------------------------------------
Link: https://www.songsterr.com/a/wsa/metallica-ride-the-lightning-tab-s248t1
-------------------------------------------------------------
Link: https://www.songsterr.com/a/wsa/alan-walker-faded-fingerstyle-tab-s408190t0
-------------------------------------------------------------
Link: https://www.songsterr.com/a/wsa/johann-sebastian-bach-bouree-tab-s390711t0
-------------------------------------------------------------
Link: https://www.songsterr.com/a/wsa/tab-s10887t0
-------------------------------------------------------------
Link: https://www.songsterr.com/a/wsa/as-i-lay-dying-through-struggle-tab-s25864t0
-------------------------------------------------------------
Link: https://www.songsterr.com/a/wsa/lynyrd-skynyrd-free-bird-tab-s21t3
-------------------------------------------------------------
Link: https://www.songsterr.com/a/wsa/judas-priest-breaking-the-law-tab-s10640t3
-------------------------------------

-------------------------------------------------------------
Link: https://www.songsterr.com/a/wsa/jason-becker-altitudes-tab-s23509t0
-------------------------------------------------------------
Link: https://www.songsterr.com/a/wsa/lamb-of-god-laid-to-rest-tab-s12062t0
-------------------------------------------------------------
Link: https://www.songsterr.com/a/wsa/duman-bu-aksam-tab-s30551t0
-------------------------------------------------------------
Link: https://www.songsterr.com/a/wsa/iron-maiden-the-trooper-tab-s338t1
-------------------------------------------------------------
Link: https://www.songsterr.com/a/wsa/death-symbolic-tab-s6352t0
-------------------------------------------------------------
Link: https://www.songsterr.com/a/wsa/cream-crossroads-tab-s242t0
-------------------------------------------------------------
Link: https://www.songsterr.com/a/wsa/lamb-of-god-redneck-tab-s24278t0
-------------------------------------------------------------
Link: https:/

-------------------------------------------------------------
Link: https://www.songsterr.com/a/wsa/yngwie-malmsteen-far-beyond-the-sun-tab-s13478t0
-------------------------------------------------------------
Link: https://www.songsterr.com/a/wsa/b-b-king-the-thrill-is-gone-tab-s2316t0
-------------------------------------------------------------
Link: https://www.songsterr.com/a/wsa/3-doors-down-kryptonite-tab-s629t0
-------------------------------------------------------------
Link: https://www.songsterr.com/a/wsa/rammstein-du-hast-tab-s13984t1
-------------------------------------------------------------
Link: https://www.songsterr.com/a/wsa/jim-croce-time-in-a-bottle-tab-s4740t1
-------------------------------------------------------------
Link: https://www.songsterr.com/a/wsa/pearl-jam-even-flow-tab-s384t0
-------------------------------------------------------------
Link: https://www.songsterr.com/a/wsa/pantera-domination-tab-s26781t0
-------------------------------------------

-------------------------------------------------------------
Link: https://www.songsterr.com/a/wsa/iron-maiden-phantom-of-the-opera-tab-s10046t0
-------------------------------------------------------------
Link: https://www.songsterr.com/a/wsa/slipknot-before-i-forget-tab-s1379t1
-------------------------------------------------------------
Link: https://www.songsterr.com/a/wsa/three-days-grace-animal-i-have-become-tab-s466t1
-------------------------------------------------------------
Link: https://www.songsterr.com/a/wsa/green-day-holiday-tab-s499t3
-------------------------------------------------------------
Link: https://www.songsterr.com/a/wsa/rammstein-deutschland-tab-s451140t0
-------------------------------------------------------------
Link: https://www.songsterr.com/a/wsa/francisco-tarrega-recuerdos-de-la-alhambra-tab-s20178t0
-------------------------------------------------------------
Link: https://www.songsterr.com/a/wsa/led-zeppelin-kashmir-tab-s288t1
---------------

...

-------------------------------------------------------------
Link: https://www.songsterr.com/a/wsa/genshin-magatsumi-takenaru-kaminomikoto-tab-s507303t0
-------------------------------------------------------------
Link: https://www.songsterr.com/a/wsa/continuous-atrocities-against-the-condemned-species-for-the-purpose-of-self-preservation-from-the-inevitable-consequences-of-playing-god-infront-the-sights-of-nature-and-existance-rage-pure-unfiltered-uncontrollable-and-unstoppable-rage-spewing-from-the-blood-vessels-causing-shaking-within-the-body-tab-s507142t1
-------------------------------------------------------------
Link: https://www.songsterr.com/a/wsa/continuous-atrocities-against-the-condemned-species-for-the-purpose-of-self-preservation-from-the-inevitable-consequences-of-playing-god-infront-the-sights-of-nature-and-existance-witnessing-every-human-act-of-disrespect-towards-their-settlement-as-the-old-man-barely-breathes-with-his-lungs-his-cries-remain-unheard-because-his-nei

-------------------------------------------------------------
Link: https://www.songsterr.com/a/wsa/highwaymen-highwayman-tab-s504427t8
-------------------------------------------------------------
Link: https://www.songsterr.com/a/wsa/seksiseppo02-riff25-tab-s504226t0
-------------------------------------------------------------
Link: https://www.songsterr.com/a/wsa/trees-of-eternity-my-requiem-tab-s504184t0
-------------------------------------------------------------
Link: https://www.songsterr.com/a/wsa/archfire-spontaneous-generation-tab-s504162t0
-------------------------------------------------------------
Link: https://www.songsterr.com/a/wsa/flux-tdc3-tab-s504153t6
-------------------------------------------------------------
Link: https://www.songsterr.com/a/wsa/flux-tdc2-tab-s504130t6
-------------------------------------------------------------
Link: https://www.songsterr.com/a/wsa/flux-tdc1-tab-s504127t6
-------------------------------------------------------------
Link: h

-------------------------------------------------------------
Link: https://www.songsterr.com/a/wsa/creatures-of-furkahn-enter-the-forest-tab-s491536t12
-------------------------------------------------------------
Link: https://www.songsterr.com/a/wsa/creatures-of-furkahn-book-tab-s491531t13
-------------------------------------------------------------
Link: https://www.songsterr.com/a/wsa/creatures-of-furkahn-creature-2-tab-s491530t2
-------------------------------------------------------------
Link: https://www.songsterr.com/a/wsa/creatures-of-furkahn-creature-1-tab-s491529t3
-------------------------------------------------------------
Link: https://www.songsterr.com/a/wsa/hellcrow-into-the-void-tab-s491362t5
-------------------------------------------------------------
Link: https://www.songsterr.com/a/wsa/hellcrow-the-depths-tab-s491357t9
-------------------------------------------------------------
Link: https://www.songsterr.com/a/wsa/alone-at-the-spire-mark-my-words-tab-s49133

-------------------------------------------------------------
Link: https://www.songsterr.com/a/wsa/pierre-bachelet-emmanuelle-tab-s488039t5
-------------------------------------------------------------
Link: https://www.songsterr.com/a/wsa/my-bloody-valentine-cigarette-in-your-bed-tab-s487912t4
-------------------------------------------------------------
Link: https://www.songsterr.com/a/wsa/tadzik-tab-s487861t3
B̲a̲n̲d̲ ̲a̲n̲d̲ ̲S̲o̲n̲g̲ ̲n̲o̲t̲ ̲f̲o̲u̲n̲d
-------------------------------------------------------------
Link: https://www.songsterr.com/a/wsa/tadzik-tab-s487862t3
B̲a̲n̲d̲ ̲a̲n̲d̲ ̲S̲o̲n̲g̲ ̲n̲o̲t̲ ̲f̲o̲u̲n̲d
-------------------------------------------------------------
Link: https://www.songsterr.com/a/wsa/tadzik-tab-s487858t3
-------------------------------------------------------------
Link: https://www.songsterr.com/a/wsa/tadzik-tab-s487859t3
B̲a̲n̲d̲ ̲a̲n̲d̲ ̲S̲o̲n̲g̲ ̲n̲o̲t̲ ̲f̲o̲u̲n̲d
-------------------------------------------------------------
Link: https://www.s

,Band,Song,Player,Technique,Difficulty,Score
0,The Beatles,Blackbird,Paul McCartney,"[Guitar, Acoustic Guitar (nylon)]",Intermediate,3/8
1,Isaac Albéniz,Asturias (Leyenda),Ýz 1,[Acoustic Guitar (nylon)],Intermediate,5/8
2,AC/DC,Back In Black,Angus Young (Lead),[Distortion Guitar],Intermediate,3/8
3,Pachelbel,Canon in D Major,NaN,[Track: Acoustic Guitar (nylon)],Intermediate,4/8
4,Led Zeppelin,"Babe, I'm Gonna Leave You",Jimmy Page,[Acoustic Guitar (steel)],Beginner,2/8
...,...,...,...,...,...,...
7281,Lemon Demon,Two Trucks,Bass Piano,[Bright Acoustic Piano],NaN,NaN
7282,Pierre Bachelet,Emmanuelle,Sitar,[],NaN,NaN
7283,My Bloody Valentine,Cigarette in Your Bed,Fuzz Bass,[Distortion Guitar],Beginner,1/8
7284,Tadzik,Красивая музыка,Bass,[Overdriven Guitar],Intermediate,3/8


In [ ]:
a = pd.read_csv(dataset_path + 'Dataset/1_Dataset_Songs_Features.csv', index_col = 'Unnamed: 0')
a.head(50)

,Band,Song,Player,Technique,Difficulty,Score
0,The Beatles,Blackbird,Paul McCartney,"['Guitar', 'Acoustic Guitar (nylon)']",Intermediate,3/8
1,Isaac Albéniz,Asturias (Leyenda),Ýz 1,['Acoustic Guitar (nylon)'],Intermediate,5/8
2,AC/DC,Back In Black,Angus Young (Lead),['Distortion Guitar'],Intermediate,3/8
3,Pachelbel,Canon in D Major,NaN,['Track: Acoustic Guitar (nylon)'],Intermediate,4/8
4,Led Zeppelin,"Babe, I'm Gonna Leave You",Jimmy Page,['Acoustic Guitar (steel)'],Beginner,2/8
5,Jimi Hendrix,Little Wing,Jimi Hendrix,['Electric Guitar (jazz) 1'],Intermediate,5/8
6,Pink Floyd,Shine On You Crazy Diamond (Parts I - IX),David Gilmour,"['Lap Steel Harmony', 'Overdriven Guitar']",Beginner,2/8
7,Kansas,Dust In The Wind,Guitar 1,['Acoustic Guitar (nylon)'],Intermediate,3/8
8,Francisco Tarrega,Capricho Arabe,Serenata para Guitarra,['Acoustic Guitar (nylon)'],Intermediate,5/8
9,Pachelbel,Canon In C,NaN,['Track: Acoustic Guitar (nylon)'],Beginner,1/8
